In [ ]:
import networkx as nx
import heapq

In [ ]:
G = nx.DiGraph()
edges = [(1, 2, 10),
         (2, 3, 1),
         (3, 4, 10),
         (1, 9, 20),
         (9, 2, 1),
         (2, 6, 1),
         (6, 8, 1),
         (8, 9, 1),
         (2, 8, 3),
         (8, 5, 15),
         (3, 5, 18),
         (5, 4, 1)]

# A 1
# B 2
# C 3
# D 4
# E 5
# F 6
# G 7
# H 8
# I 9

G.add_weighted_edges_from(edges)

In [ ]:
def dijkstra(graph, src, dest):

  heap = [(0, src)]
  shortest_path = {node: float('inf') for node in graph}
  shortest_path[src] = 0

  while heap:
    cost, node = heapq.heappop(heap)

    if cost > shortest_path[node]:
      continue

    for neighbor, data in graph[node].items():
      new_cost = cost + data['weight']

      if new_cost < shortest_path[neighbor]:
        shortest_path[neighbor] = new_cost
        heapq.heappush(heap, (new_cost, neighbor))

      if neighbor == node:
        return shortest_path[dest]
        
  return shortest_path[dest]

In [ ]:
def reverse(graph):
  Gr = nx.DiGraph()
  # for node in graph.nodes():
  #   Gr.add_node(node)

  Gr.add_edges_from((v,u,d) for u,v,d in graph.edges(data=True))
  return Gr

In [ ]:
GR = reverse(G)

In [ ]:
dist = []
isSettled = []
PQ = []
parent = []

In [ ]:
def ConstructPartialSPT(reverse_graph, src, dest):
  if isSettled[dest] == True:
    return dist[dest]

  while PQ:
    u = heapq.heappop(PQ)
    isSettled[u] = True
    
    if u == dest:
      return dist[u]
    
    for u_neighbor, data in G[u] and isSettled[u_neighbor] == False:
      if dist[u] + data['weight'] < dist[u_neighbor]:
        dist[u_neighbor] = dist[u] + data['weight']
        parent[u_neighbor] = u
        heapq.heappush(PQ, u_neighbor)
        
  return float('inf')

In [ ]:
ConstructPartialSPT(GR, 4, 1)
dist